To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


**NEW** Unsloth now supports training the new **gpt-oss** model from OpenAI! You can start finetune gpt-oss for free with our **[Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)**!

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### **1. Setup: Install Unsloth and Dependencies**

Unsloth's installation is specific to your GPU's architecture. The following command is optimized for Google Colab GPUs (like T4, V100).

In [ ]:
# Install Unsloth for Colab notebooks
#!pip install "unsloth[colab-new] @ git+https://github.com/unsloth/unsloth.git"
!pip install -q transformers datasets peft trl

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### **2. Prepare the Dataset**

We'll create our small, high-quality Luganda sentiment dataset. Unsloth works directly with Hugging Face `Dataset` objects.

In [ ]:
from datasets import load_dataset

# Define the path to your uploaded JSON file
file_path = "sentiment-luganda.json"

# Load the dataset directly from the JSON file
full_dataset = load_dataset('json', data_files=file_path)

# The dataset is loaded as a DatasetDict, we'll work with the 'train' split
dataset = full_dataset['train']

# --- Formatting function ---
def format_instruction(example):
    instruction = f"Analyze the sentiment of this Luganda sentence. The sentence is: \"{example['sentence']}\". The label is:"
    response = example['label']
    messages = [
        {"role": "user", "content": instruction},
        {"role": "model", "content": response}
    ]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": formatted_prompt}  # return as dict with "text" key

# --- Map before splitting ---
dataset = dataset.map(format_instruction, remove_columns=dataset.column_names)


# --- BEST PRACTICE: Split the dataset into training and testing sets ---
# This helps you evaluate how well your model generalizes to unseen data.
split_dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

# Rename the splits for clarity
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

print("Dataset loaded and split successfully:")
print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}\n")

print("Example from the training set:")
print(train_dataset[0])

**If you do not have a training dataset, you can use this one**

In [ ]:
from datasets import Dataset

# Our custom Luganda dataset
data = {
    'omutwe': [ # 'omutwe' means 'sentence' or 'text'
        "Njagala nnyo filimu eno.", "Emmere ebadde ewooma.", "Olunaku luno lwali lwa ssanyu.",
        "Sitwagala n'akatono filimu eno.", "Emmere ebadde mbi nnyo.", "Nali munakuwavu nnyo.",
        "Leero olunaku lwa Lwakusatu.", "Mmotoka eno ya myaka kkumi.", "Enkuba etonnya.",
        "Yazaalidwa mu mwezi gwa Mukutulansanja.", "Anonya mulimu.", "Olunaku lwabadde lwa bulijjo.",
        "Ekitabo kino kinyuma nnyo!", "Babadde bantu balungi nnyo."
    ],
    'omuwendo': [ # 'omuwendo' means 'label' or 'value'
        "Kirungi", "Kirungi", "Kirungi",
        "Kibi", "Kibi", "Kibi",
        "Kya bulijjo", "Kya bulijjo", "Kya bulijjo",
        "Kya bulijjo", "Kya bulijjo", "Kya bulijjo",
        "Kirungi", "Kirungi"
    ]
}

# Convert to a Hugging Face Dataset object
luganda_sentiment_dataset = Dataset.from_dict(data)

print("Dataset created successfully:")
print(luganda_sentiment_dataset[0])

* * *

### **3. Load the Model Using Unsloth**

This is where the magic of Unsloth begins. We'll use `FastLanguageModel` to load our model. Notice how much simpler it is—Unsloth handles the quantization, data types, and device mapping automatically under the hood.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "CraneAILabs/ganda-gemma-1b",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

### **4. Fine-Tuning with the TRL Trainer**

The training process remains almost identical to the standard approach, as Unsloth integrates seamlessly with the `SFTTrainer` from the `trl` library. We just need to define our formatting function and training arguments.

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

In [ ]:
# --- Formatting func for Unsloth (wrap in list here) ---
def formatting_func(row):
    return [row["text"]]

In [ ]:
# --- Train ---
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func,
    max_seq_length=512,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=False,
        bf16=False,
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

In [ ]:

# Start training! Keep an eye on the speed (tokens/sec).
print("Starting the Unsloth fine-tuning process...")
trainer.train()
print("Fine-tuning complete!")

* * *

### **5. Inference with the Fine-Tuned Unsloth Model**

Now, let's use our specialized model for prediction. Unsloth models are fully compatible with Hugging Face pipelines and `.generate()` methods.

In [ ]:
# Create an inference prompt
test_sentence = "Siwagira ttiimu eno." # "I do not support this team."
instruction = f"Analyze the sentiment of this Luganda sentence. The sentence is: \"{test_sentence}\". The label is:"
messages = [{"role": "user", "content": instruction}, {"role": "model", "content": ""}]

In [ ]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=5)
generated_tokens = outputs[:, inputs.input_ids.shape[1]:]
final_response = tokenizer.decode(generated_tokens[0], skip_special_tokens=True).strip()

print(f"Sentence: '{test_sentence}'")
print(f"Predicted Sentiment: {final_response}")

In [ ]:
# --- Another Test ---
test_sentence_pos = "Oluyimba luno lunyuma nnyo." # "This song is very nice."
instruction = f"Analyze the sentiment of this Luganda sentence. The sentence is: \"{test_sentence_pos}\". The label is:"
messages = [{"role": "user", "content": instruction}, {"role": "model", "content": ""}]

In [ ]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=5)
generated_tokens = outputs[:, inputs.input_ids.shape[1]:]
final_response = tokenizer.decode(generated_tokens[0], skip_special_tokens=True).strip()

print(f"Sentence: '{test_sentence_pos}'")
print(f"Predicted Sentiment: {final_response}")


### **6. Saving the Model**

You can save the trained LoRA adapters in the standard Hugging Face format, making them portable and easy to use elsewhere.

In [ ]:

# Save the trained LoRA adapters
model.save_pretrained("luganda_sentiment_lora")

print("Model adapters saved to 'luganda_sentiment_lora'")

# You can also push them to the Hugging Face Hub

In [ ]:
model.push_to_hub("yourusername/luganda-sentiment-gemma-unsloth", token="hf_...")

tokenizer.push_to_hub("yourusername/luganda-sentiment-gemma-unsloth", token="hf_..")

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
